# Análise de Comportamente de Consumidores

Nesse trabalho iremos utilizar dados de e-commerce para analisar o comportamento dos clientes, com objetivo de identificar padrões de consumo e grupos que podem presentes nos dados.

## Problema

Sua empresa tem uma verba de Marketing pequena para esse ano, que no máximo deve conseguir realizar uma campanha para 1000 consumidores. Seu trabalho como analista de dados é identificar padrões de consumo nos dados e segmentar a base de clientes em grupos com características úteis para equipe de marketing.

## Exercício

* A análise deve ser realizada para os consumidores
* Identifique grupos com características específicas nos dados, defina quais características são úteis com base em dois princípios **Fidelidade** e **Valor**.
* Apresente a justificativa do grupo identificado na análise, quais características foram levadas em conta e uma sugestão de campanha de marketing


![Cota](https://cdn.bpodataentryhelp.com/sites/default/files/u108/Tricks%20for%20Best%20User%20Experience%20Via%20E-commerce%20Data%20Entry%20Form.jpg)

## Dataset

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.


 * |-- **InvoiceNo**: Código do pedido
 * |-- **StockCode**: Código da Loja
 * |-- **Description**: Descrição do Pedido
 * |-- **Quantity**: Quantidade
 * |-- **InvoiceDate**: Data do Pedido 
 * |-- **UnitPrice**: Preço do Item
 * |-- **CustomerID** ID do consumidor
 * |-- **Country**: País
 

* https://www.kaggle.com/carrie1/ecommerce-data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', 500)

## Baixando o Dataset

In [ ]:
!wget https://storage.googleapis.com/ms_geral/customer_dataset.csv

## Leitura do Dataset

In [ ]:
df = pd.read_csv('./customer_dataset.csv', encoding="ISO-8859-1", dtype={'CustomerID': str}) #Leitura do dataset

df = df.dropna() #Descartando valores nulos

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#Filtrando apenas valores maiores que zero
df = df[df.Quantity >= 0]

In [ ]:
#Criando uma nova feature para dataset

df['TotalPaid'] = df.Quantity * df.UnitPrice

df.head()

In [ ]:
df.describe()

### Criação do dataset por Consumidor, agregando informações de consumo.

**Dica: ** Crie mais variáveis que caracterizem o perfil de consumo do cliente, quanto mais features melhor as análise de clientes. Tente usar clusterização ou trabalhar com medidas de tendência central para separar os grupos.

In [ ]:
df.InvoiceDate  = pd.to_datetime(df.InvoiceDate)

df.head()

In [ ]:
#Agrupando por CustomerID

agg_func = {
    'InvoiceDate':['min'],
    'TotalPaid':['sum'],
    'CustomerID':['count']
}

df_customer = df.groupby('CustomerID').agg(agg_func)

df_customer.columns= ['_'.join(col).strip() for col in df_customer.columns.values] #Apenas ajustando os nomes das colunas

df_customer = df_customer.reset_index()

df_customer.head()

In [ ]:
#Dias como consumidor

hoje= pd.Timestamp.today()

df_customer['DaysAsCustomer'] = hoje - df_customer.InvoiceDate_min

df_customer['DaysAsCustomer'] = df_customer['DaysAsCustomer'].apply(lambda x: x.days)
df_customer.head()

In [ ]:
#Mais algumas features

df_customer['MeanValueByProduct'] = df_customer.TotalPaid_sum/ df_customer.CustomerID_count

In [ ]:
#Selecionando colunas para treinar o modelo kmeans
df_input = df_customer.drop('InvoiceDate_min', axis=1)

#Criando o modelo
kmeans = KMeans(n_clusters = 12, random_state=42)
kmeans.fit(df_input)

In [ ]:
labels = kmeans.predict(df_input)

In [ ]:
df_customer['Labels'] = labels

In [ ]:
df_customer.head()

### Agrupando os clientes nos clusters identificados:

In [ ]:
df_clusters = df_customer.groupby('Labels').agg('first') #Pegando apenas o primeiro elemento de cada entrada no dataset

df_clusters = df_clusters.reset_index()

In [ ]:
df_clusters.head()